In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Qwen2MoeForCausalLM
import torch
import torch.nn.functional as F

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
chat_model_name = "Qwen/Qwen2.5-1.5B-Instruct"
math_model_name = "Qwen/Qwen2.5-Math-1.5B-Instruct"
coder_model_name = "Qwen/Qwen2.5-Coder-1.5B-Instruct"
# tokenizer = AutoTokenizer.from_pretrained(chat_model_name)
# chat_model = AutoModelForCausalLM.from_pretrained(chat_model_name, device_map="auto")
# math_model = AutoModelForCausalLM.from_pretrained(math_model_name, device_map="auto")
# coder_model = AutoModelForCausalLM.from_pretrained(coder_model_name, device_map="auto")

In [4]:
from model import Qwen2_5MoEForCausalLM

In [5]:
moe_model = Qwen2_5MoEForCausalLM.merge_models(
    model_name_or_path=[
        "Qwen/Qwen2.5-1.5B-Instruct",
        "Qwen/Qwen2.5-Math-1.5B-Instruct",
        "Qwen/Qwen2.5-Coder-1.5B-Instruct",
    ]
)

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


len(experts) 3
len(moe_model.model.layers) 28


In [6]:
moe_model

Qwen2_5MoEForCausalLM(
  (model): Qwen2_5MoEModel(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2_5MoEDecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (router): Qwen2_5MoEExpertRouter(
          (router): Linear(in_features=1536, out_features=3, bias=False)
        )
        (experts): ModuleList(
          (0-2): 3 x Qwen2MLP(
            (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
            (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
            (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
            (act_fn): SiLU()
          )
        )
        (in

In [7]:
# try forward
input_ids = torch.randint(0, 1000, (2, 10))
attention_mask = torch.ones((2, 10))
position_ids = torch.arange(10).unsqueeze(0).expand_as(input_ids)
# head_mask = torch.ones((2, 10))
outputs = moe_model(
    input_ids=input_ids,
    attention_mask=attention_mask,
    position_ids=position_ids,
)
print(outputs)

top_k_probs tensor([[0.3624],
        [0.3561],
        [0.3706],
        [0.3847],
        [0.3648],
        [0.3381],
        [0.3527],
        [0.3755],
        [0.3774],
        [0.3739],
        [0.3840],
        [0.4066],
        [0.3692],
        [0.3459],
        [0.4001],
        [0.3671],
        [0.3556],
        [0.3417],
        [0.3653],
        [0.3892]], grad_fn=<TopkBackward0>)
top_k_indices tensor([[0],
        [1],
        [0],
        [0],
        [2],
        [1],
        [0],
        [0],
        [1],
        [0],
        [2],
        [2],
        [2],
        [2],
        [0],
        [2],
        [0],
        [2],
        [0],
        [0]])
top_k_probs tensor([[0.7221],
        [0.8156],
        [0.7090],
        [0.6392],
        [0.4805],
        [0.4440],
        [0.6823],
        [0.7380],
        [0.5012],
        [0.7330],
        [0.8631],
        [0.8201],
        [0.7591],
        [0.6105],
        [0.5093],
        [0.7112],
        [0.7275],
        [

top_k_probs tensor([[0.3479],
        [0.3626],
        [0.4870],
        [0.3728],
        [0.4406],
        [0.3491],
        [0.4947],
        [0.4269],
        [0.4907],
        [0.3878],
        [0.7022],
        [0.3613],
        [0.4125],
        [0.4997],
        [0.4274],
        [0.4590],
        [0.4047],
        [0.3886],
        [0.5129],
        [0.4294]], grad_fn=<TopkBackward0>)
top_k_indices tensor([[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [2],
        [0],
        [0],
        [2],
        [0],
        [2],
        [0],
        [1],
        [1],
        [1],
        [0]])
top_k_probs tensor([[0.4016],
        [0.4274],
        [0.3528],
        [0.3992],
        [0.3422],
        [0.4753],
        [0.4264],
        [0.3736],
        [0.4087],
        [0.4031],
        [0.5107],
        [0.3618],
        [0.4542],
        [0.4135],
        [0.4229],
        [0.4120],
        [0.3872],
        [

In [9]:
# calculate params
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Total parameters:", count_parameters(moe_model) / 1e6, "M")

Total parameters: 3855.953408 M


In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [14]:

prompt = "Please reason step by step, and put your final answer within \\boxed{}. Find the value of $x$ that satisfies the equation $4x^2+6 = 6x+4$."

# CoT
messages = [
    # {"role": "system", "content": "Please reason step by step, and put your final answer within \\boxed{}."},
    {"role": "user", "content": prompt}
]

# TIR
# messages = [
#     {"role": "system", "content": "Please integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}."},
#     {"role": "user", "content": prompt}
# ]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print(text)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=1024
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


<｜begin▁of▁sentence｜><｜User｜>Please reason step by step, and put your final answer within \boxed{}. Find the value of $x$ that satisfies the equation $4x^2+6 = 6x+4$.<｜Assistant｜><think>



"Alright, so I've got this equation here: 4x squared plus 6 equals 6x plus 4. Hmm, okay. I need to find the value of x that satisfies this. Let me think about how to approach this. It looks like a quadratic equation, right? Because there's an x squared term. So, maybe I can rearrange it into the standard quadratic form and then solve for x.\n\nFirst, let me write down the equation again to make sure I have it right: 4x² + 6 = 6x + 4. Yeah, that's correct. So, the left side is a quadratic expression, and the right side is a linear expression. To solve for x, I should get all terms on one side so that I have zero on the other side. That way, I can factor or use the quadratic formula if needed.\n\nLet me subtract 6x and 4 from both sides to move them to the left. That should help. So, subtracting 6x and 4 from both sides gives me:\n\n4x² + 6 - 6x - 4 = 0.\n\nSimplify the constants and like terms. 6 minus 4 is 2, so that simplifies to:\n\n4x² - 6x + 2 = 0.\n\nHmm, okay. Now, I have a quadr

In [15]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 128.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 131.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 179.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2


In [20]:
from datasets import load_dataset

facebook_dataset = load_dataset("facebook/natural_reasoning", split="train")

In [ ]:
def change_box_by_reference_answer(batch):
    # 'question', 'reference_answer', 'responses'
    import re
    # \\\boxed{}
    final_answer_regex = re.compile(r"\\boxed{([^}]*)}")
    ignore_data = []
    for question, reference_answer, responses in zip(batch['question'], batch['reference_answer'], batch['responses']):
        # find the last \\boxed{} in the responses
        try:
            rex = re.compile(r"\\boxed{([^}]*)}")
            # find the last \\boxed{} in the responses
            last_boxed = None
            if len(responses) == 0 or responses is None:
                ignore_data.append(True)
                continue
            for response in responses:
                match = rex.search(response)
                if match:
                    last_boxed = match.group(1)
            if last_boxed is not None:
                ignore_data.append(False)
                if last_boxed != reference_answer and reference_answer != "" and reference_answer is not None:
                    # replace the last \\boxed{} with the reference_answer
                    for i, response in enumerate(responses):
                        responses[i] = rex.sub(f"\\boxed{{{reference_answer}}}", response)
            else:
                ignore_data.append(True)
        except Exception as e:
            print(e, question, reference_answer, responses)
            ignore_data.append(True)
    batch['ignore_data'] = ignore_data
    return batch

facebook_dataset = facebook_dataset.map(change_box_by_reference_answer, batched=True)

DatasetDict({
    train: Dataset({
        features: ['question', 'reference_answer', 'responses'],
        num_rows: 1145824
    })
})

In [ ]:
glaive_dataset = load_dataset("glaiveai/reasoning-v1-20m")